In [34]:
import math
import torch
from torch import FloatTensor

In [79]:
def generate_disc_set(nb):
    input = FloatTensor(nb, 2).uniform_(0, 1)
    target = torch.norm(input,2,1) < math.sqrt(1/(2*math.pi))
    return input, target.float()

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

NameError: name 'Variable' is not defined

In [52]:
# Simple structure for a Module

class Module ( object ) :
    def forward ( self , * input ) :
        raise NotImplementedError
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
    def param ( self ) :
        return []

In [80]:
# Module for MSEloss

class MSEloss( object ):
    def forward ( self , input, target ) :
        return input.sub(target).pow(2).sum()
    
    def backward ( self , input, target ) :
        return 2 * (input.sub(target))    

In [81]:
criterion = MSEloss()

In [85]:
loss = criterion.forward(train_target, test_target)
dloss = criterion.backward(train_target, test_target)